In [ ]:
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import json
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from seleniumwire.utils import decode

In [ ]:
chrome_options = Options()
chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
chrome_options.add_argument('--auto-open-devtools-for-tabs')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable_encoding')

driver = webdriver.Chrome(options=chrome_options)

driver.execute_cdp_cmd("Network.enable", {})

wait = WebDriverWait(driver, 120)

In [ ]:
#Get Data
import json
import inspect

def read_data(route):
    try:
        with open(f'{route}.json', 'r', encoding='utf-8') as file:
            return json.load(file)
    except Exception as e:
        print(f'Error al decodificar archivo: {route}')
        return []

def filter_data(data, name):
    filtered_list = list(filter(lambda item: 'ean' in item and item['ean'] != "" and item['ean'] != "-" and 'price' in item and item['price'] != "", data))
    print(f'{len(filtered_list)} Productos con ean y price -> En {name} de {len(data)}')
    #return filtered_list

jardinzoo_dogs = read_data('jardinzoo/dogsProducts')
jardinzoo_cats = read_data('jardinzoo/catsProducts')


In [ ]:
dogs_without_ean = list(filter(lambda item: item['ean'] == "" or item['price'] == "", jardinzoo_dogs))
cats_without_ean = list(filter(lambda item: item['ean'] == "" or item['price'] == "", jardinzoo_cats))

In [ ]:
from selenium.common.exceptions import NoSuchElementException

def go_to_product(product):
    wait = WebDriverWait(driver, 120)
    driver.get(product['product_url'])
    
    try:
        product_tab_reference = wait.until(
            EC.presence_of_element_located(
                (By.XPATH, "//a[contains(text(), 'Detalles del producto')]")
            )
        )
        product_tab_reference.click()
    except NoSuchElementException:
        print(f"No se encontró ean oculto en {product['title']}")
        
    try:
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.product-reference')))
        product_reference = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.product-reference')))
        product['ean'] = product_reference.find_element(By.CSS_SELECTOR, 'span').text
    except NoSuchElementException:
        print(f"La referencia no se encontró en el producto: {product['title']}")
        product['ean'] = ''

    try:
        product['brand'] = driver.find_element(By.CSS_SELECTOR, '.product-manufacturer').find_element(By.CSS_SELECTOR, 'a').text
    except NoSuchElementException:
        print(f"La marca no se encontró en el producto {product['title']}")
        product['brand'] = ''
    
    product['price'] = driver.find_element(By.CSS_SELECTOR, '.current-price').find_element(By.CSS_SELECTOR, 'span').get_attribute('content')
    product['currency'] = 'CLP'

for product in cats_without_ean:
    try:
        go_to_product(product)
    except Exception as e:
        print(f"Error inesperado producto {product['title']}: {e}")

In [ ]:
print(cats_without_ean)

In [ ]:
with open(f"cats_without_ean.json", 'w', encoding='utf-8') as json_file:
    json.dump(cats_without_ean, json_file, ensure_ascii=False, indent=4)

print("Archivo JSON generado correctamente.")